In [ ]:
import os
import csv
import gymnasium as gym
from stable_baselines3 import DQN
import requests
import json
import numpy as np

#OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY", "API_KEY")

def short_specific_prompt(trajectory, total_reward):
    return (
        f"Provide a short and precise analysis of why the agent failed in the Lunar Lander environment.\n"
        f"State vector details: [x, y, x_velocity, y_velocity, angle, angular_velocity, left_leg_contact, right_leg_contact]\n\n"
        f"Here are the last 50 steps of the trajectory (state, action, reward at each step):\n{trajectory}\n\n"
        f"Total reward: {total_reward}\n\n"
        f"Please explain the main reasons for failure."
    )

def long_detailed_prompt(trajectory, total_reward, env_description):
    return (
        f"Provide a detailed analysis of the agent's failure in the Lunar Lander environment.\n"
        f"Environment details:\n{env_description}\n\n"
        f"Here are the last 50 steps of the trajectory:\n{trajectory}\n\n"
        f"Total reward: {total_reward}\n\n"
        f"Please include specific factors (position, velocity, angle, leg contacts, etc.) that contributed to this failure."
    )

def short_specific_prompt_modified(trajectory, total_reward):
    return (
        f"Provide a short and precise analysis of why the agent failed in the environment.\n"
        f"State vector details: [right_leg_contactx, left_leg_contact, angular_velocitym angle, y_velocity, x_velocity, y, x]\n\n"
        f"Here are the last 50 steps of the trajectory (state, action, reward at each step):\n{trajectory}\n\n"
        f"Total reward: {total_reward}\n\n"
        f"Please explain the main reasons for failure."
    )

def generate_explanation(model_id, prompt):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {"Authorization": f"Bearer {OPENROUTER_API_KEY}", "Content-Type": "application/json"}
    data = {"model": model_id, "messages": [{"role": "user", "content": prompt}], "temperature": 0.7, "top_p": 0.9}
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data), timeout=30)
        if response.status_code != 200:
            return f"Error: HTTP {response.status_code} - {response.text[:200]}"
        return response.json()['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error: Request failed: {e}"

def generate_reference(final_state, total_reward):
    x, y, x_vel, y_vel, angle, ang_vel, left_leg, right_leg = final_state

    # Combine all possible reasons according to the heuristic thresholds
    reason = ""

    if abs(x_vel) > 0.6 or abs(y_vel) > 0.6:
        reason += " The agent crashed because its horizontal or vertical velocity was too high during descent."

    if abs(angle) > 0.4 or abs(ang_vel) > 0.5:
        reason += " The agent became too tilted or rotated too quickly, causing an unstable descent and crash."

    if left_leg < 0.5 and right_leg < 0.5:
        reason += " The agent failed because neither landing leg made ground contact before impact."

    if abs(x) > 0.3:
        reason += " The agent drifted too far horizontally from the landing pad, leading to an unsafe touchdown."

    # This is only if no other reason was found
    if len(reason) == 0:
        reason += " The agent crashed due to instability or excessive movement near the surface."
    
    return "The agent crashed for one the following reasons:" + reason

env = gym.make("LunarLander-v3")
model = DQN.load("Trained_LL", env=env, device="cpu")

env_description = env.unwrapped.__doc__
ACTION_MAP = {0: "Do nothing", 1: "Fire left engine", 2: "Fire main engine", 3: "Fire right engine"}

models = [
    ("openai/gpt-5-chat", "GPT-5"),
    ("meta-llama/llama-3.2-11b-vision-instruct", "Llama-3"),
    ("deepseek/deepseek-r1-0528", "DeepSeek")
]

results = []
episodes = 200

for ep in range(episodes):
    obs, _ = env.reset()
    done = False
    trajectory = []
    total_reward = 0.0
    steps = 0

    while not done and steps < 1000:
        action, _ = model.predict(obs, deterministic=True)
        new_obs, reward, terminated, truncated, _ = env.step(int(action))
        total_reward += reward
        trajectory.append({
            "state": [float(x) for x in obs],
            "action": ACTION_MAP[int(action)],
            "reward": float(reward)
        })
        obs = new_obs
        done = terminated or truncated
        steps += 1

    if not trajectory:
        continue

    if trajectory[-1]['reward'] <= -100:
        outcome = "Failure"
    else:
        outcome = "Success"

    if outcome == "Failure":
        last_steps = list(enumerate(trajectory[-50:], start=len(trajectory) - len(trajectory[-50:]) + 1))

        final_state = np.array(trajectory[-1]['state'])
        reference_text = generate_reference(final_state, total_reward)

        for prompt_func, prompt_name in [
            (short_specific_prompt, "ShortSpecific"),
            (long_detailed_prompt, "LongDetailed"),
            (short_specific_prompt_modified, "ShortSpecificModified"),
        ]:
            if "Modified" in prompt_name:
                # Modified LL
                trajectory_text = "\n".join(
                    f"Step {idx}: State={step['state'][::-1]}, Action={step['action']}, Reward={step['reward']}"
                    for idx, step in last_steps
                )
            else:
                # Unmodified LL
                trajectory_text = "\n".join(
                    f"Step {idx}: State={step['state']}, Action={step['action']}, Reward={step['reward']}"
                    for idx, step in last_steps
                )

            if prompt_name.startswith("ShortSpecific"):
                prompt = prompt_func(trajectory_text, total_reward)
            else:
                prompt = prompt_func(trajectory_text, total_reward, env_description)

            for model_id, model_name in models:
                explanation = generate_explanation(model_id, prompt)
                results.append([
                    ep + 1, model_name, prompt_name,
                    prompt, reference_text,
                    trajectory_text, total_reward, outcome,
                    explanation
                ])

with open("LL_LLM_Explanations.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Episode", "LLM", "PromptType",
        "Prompt", "Reference",
        "Trajectory", "TotalReward", "Outcome",
        "Explanation"
    ])
    writer.writerows(results)